<a href="https://colab.research.google.com/github/Huni1i1/Running-Posture-Correction-AI/blob/main/VideoPose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Posture Correction AI

# Environment Setup

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# !nvidia-smi

Mounted at /content/drive


In [ ]:
# Install Detectron2
%cd /content/
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
!pip install -r requirements.txt
!python setup.py install
!pip install git+https://github.com/facebookresearch/fvcore.git

In [2]:
# Clone VideoPose3D
!git clone https://github.com/facebookresearch/VideoPose3D

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121 (from 1)
Receiving objects: 100% (121/121), 9.53 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
# Download pre-trained model(Human3.6M)
!mkdir checkpoint
%cd checkpoint
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_cpn.bin

/content/checkpoint
--2024-10-05 06:32:09--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_cpn.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.33.183.33, 13.33.183.115, 13.33.183.29, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.33.183.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67889963 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_cpn.bin’

pretrained_h36m_cpn 100%[===================>]  64.74M  19.4MB/s    in 4.1s    

2024-10-05 06:32:14 (15.7 MB/s) - ‘pretrained_h36m_cpn.bin’ saved [67889963/67889963]



# Test

In [4]:
# Setup test
TEST_WEIGHTS = "/content/checkpoint/pretrained_h36m_cpn.bin"
PATH_TEST    = "/content/drive/MyDrive/running_dataset"

In [10]:
# Test(Human3.6M)
%cd /content/VideoPose3D
!python run.py -arc 3,3,3,3,3 -c checkpoint --evaluate TEST_WEIGHTS --dataset PATH_TEST

/content/VideoPose3D
Namespace(dataset='PATH_TEST', keypoints='cpn_ft_h36m_dbb', subjects_train='S1,S5,S6,S7,S8', subjects_test='S9,S11', subjects_unlabeled='', actions='*', checkpoint='checkpoint', checkpoint_frequency=10, resume='', evaluate='TEST_WEIGHTS', render=False, by_subject=False, export_training_curves=False, stride=1, epochs=60, batch_size=1024, dropout=0.25, learning_rate=0.001, lr_decay=0.95, data_augmentation=True, test_time_augmentation=True, architecture='3,3,3,3,3', causal=False, channels=1024, subset=1, downsample=1, warmup=1, no_eval=False, dense=False, disable_optimizations=False, linear_projection=False, bone_length_term=True, no_proj=False, viz_subject=None, viz_action=None, viz_camera=0, viz_video=None, viz_skip=0, viz_output=None, viz_export=None, viz_bitrate=3000, viz_no_ground_truth=False, viz_limit=-1, viz_downsample=1, viz_size=5)
Loading dataset...
Traceback (most recent call last):
  File "/content/VideoPose3D/run.py", line 49, in <module>
    raise KeyEr

In [ ]:
# Visualization(MP4)
!python run.py -k cpn_ft_h36m_dbb -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin --render --viz-subject S11 --viz-action Walking --viz-camera 0 --viz-video "/path/to/videos/S11/Videos/Walking.54138969.mp4" --viz-output output.gif --viz-size 3 --viz-downsample 2 --viz-limit 60

In [6]:
# Visualization(MP4)
%cd /content/VideoPose3D
!python run.py -arc 3,3,3,3,3 -c checkpoint --evaluate TEST_WEIGHTS --viz-subject S11 --viz-action Walking --viz-camera 0 --viz-video PATH_TEST+"video1.mp4" --viz-output output.mp4 --viz-size 5 --viz-downsample 1

/content/VideoPose3D
Namespace(dataset='h36m', keypoints='cpn_ft_h36m_dbb', subjects_train='S1,S5,S6,S7,S8', subjects_test='S9,S11', subjects_unlabeled='', actions='*', checkpoint='checkpoint', checkpoint_frequency=10, resume='', evaluate='TEST_WEIGHTS', render=False, by_subject=False, export_training_curves=False, stride=1, epochs=60, batch_size=1024, dropout=0.25, learning_rate=0.001, lr_decay=0.95, data_augmentation=True, test_time_augmentation=True, architecture='3,3,3,3,3', causal=False, channels=1024, subset=1, downsample=1, warmup=1, no_eval=False, dense=False, disable_optimizations=False, linear_projection=False, bone_length_term=True, no_proj=False, viz_subject='S11', viz_action='Walking', viz_camera=0, viz_video='PATH_TEST+video1.mp4', viz_skip=0, viz_output='output.mp4', viz_export=None, viz_bitrate=3000, viz_no_ground_truth=False, viz_limit=-1, viz_downsample=1, viz_size=5)
Loading dataset...
Traceback (most recent call last):
  File "/content/VideoPose3D/run.py", line 41, 